In [2]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from cape_webpage_parser import get_parsed_rows_cape
from ucsd_triton_sso import initiate_sso_login
import csv
from sys import stdout
from os import path

In [10]:
options = Options()
options.headless = False
driver = Firefox(options=options, executable_path=r'./../geckodriver')
driver.get("http://cape.ucsd.edu/responses/Results.aspx")
print ("Headless Firefox Initialized")

Headless Firefox Initialized


In [11]:
initiate_sso_login(driver)

====================== Initiating a SSO login ====================


In [13]:
driver.find_element_by_id('main-content').find_element_by_name('_eventId_proceed').click()

In [15]:
# we expect a security warning   
# try:
WebDriverWait(driver, 120).until(EC.alert_is_present(),
                               'Timed out waiting for a successful SSO login. Check credentials?')
#Switch the control to the Alert window
alert = driver.switch_to.alert

#Retrieve the message on the Alert window
msg = alert.text

#use the accept() method to accept the alert
alert.accept()
print(f"Alert Accepted:\n{msg}")

# except NoAlertPresentException:
#     raise 

# except TimeoutException:
#     pass

Alert Accepted:
The information you have entered on this page will be sent over an insecure connection and could be read by a third party.

Are you sure you want to send this information?


In [16]:
# wait for cape page to load after SSO
try:
    status = WebDriverWait(driver, 30, poll_frequency=0.5).until(lambda x: x.find_element_by_id('ctl00_ContentPlaceHolder1_ddlDepartments'))
except:
    assert False, 'Timed out waiting for SSO -> CAPE webpage loading'

In [17]:
select_dept = Select(driver.find_element_by_id('ctl00_ContentPlaceHolder1_ddlDepartments'))
departments = [o.text for o in select_dept.options if o.text != "Select a Department"]

In [7]:
# select departments one at a time and create their csv
for dept in departments[:2]:
    dept_keyword = dept.split()[0]
    f = open(f"./cape_{dept_keyword}_auto.csv", 'w')
    writer = csv.writer(f)

    select_dept.select_by_visible_text(dept)
    # click on submit button
    driver.find_element_by_id('ctl00_ContentPlaceHolder1_btnSubmit').click()

    # there's a small delay between click and this element getting displayed
    # returns true
    status = WebDriverWait(driver, 30, poll_frequency=0.5).\
                until(lambda x: x.find_element_by_id('ctl00_ContentPlaceHolder1_UpdateProgress1').is_displayed())

    # shoud return false now
    status = WebDriverWait(driver, 30, poll_frequency=0.5).\
                until_not(lambda x: x.find_element_by_id('ctl00_ContentPlaceHolder1_UpdateProgress1').is_displayed())
    assert status == False, "Page is still loading!"

    rows = get_parsed_rows_cape(driver.page_source)
    if (len(rows) == 0):
        print(f"Didn't find any valid row for {dept_keyword}!")
    else:
        for row in rows:
            writer.writerow(row)
        print(f"Successfully parsed {dept_keyword} results({len(rows)} items)")

    f.close()

Successfully parsed ANTH results(791 items)
Successfully parsed BENG results(777 items)


In [8]:
driver.quit()

In [22]:
'{s:{c}^{n}}'.format(s='dog',n=10,c=' ')

'   dog    '

In [74]:
'ANTH' in ['ANTH sdfsdfsf']

False

In [75]:
['ANTH sdfsdfsf'].index('ANTH sdfsdfsf')

0